SAP Credit Card Fraud Prediction
About the dataset
The dataset published by Kaggle in  2016-11-03  https://www.kaggle.com/mlg-ulb/creditcardfraud .
The datasets contains transactions made by credit cards in September 2013 by european cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.


Before we start we training the fraud detection module you need to prepare the dataset to be ready for the training stage. This process is know as data engineering and usually it can consume %80 of the machine learning project timeframe.


Start by loading the data from S3 to datafram in sagemaker.


In [90]:
from sagemaker import get_execution_role
role = get_execution_role()

In [138]:
Bucket_name='sap-on-aws-ai-ml-demo-ireland'
Prefix_name='csv/'
conn = boto3.client('s3')
contents = conn.list_objects(Bucket=Bucket_name, Prefix=Prefix_name)['Contents']
contents =contents[1:]
dfs = []
for f in contents:
    file_part_name=(f['Key'])
    file_part_name_full_name= Bucket + "/"+file_part_name
    file_part_name_full_name = "s3://"+file_part_name_full_name
    print(file_part_name_full_name) 
    dfs.append(pd.read_csv(file_part_name_full_name))
    
frame = pd.concat(dfs, axis=0, ignore_index=True)
frame= frame.drop(['recordid','partition_0'], axis=1)

s3://sap-on-aws-ai-ml-demo-ireland/csv/run-1608553911056-part-r-00000
s3://sap-on-aws-ai-ml-demo-ireland/csv/run-1608553911056-part-r-00001
s3://sap-on-aws-ai-ml-demo-ireland/csv/run-1608553911056-part-r-00002
s3://sap-on-aws-ai-ml-demo-ireland/csv/run-1608553911056-part-r-00003
s3://sap-on-aws-ai-ml-demo-ireland/csv/run-1608553911056-part-r-00004
s3://sap-on-aws-ai-ml-demo-ireland/csv/run-1608553911056-part-r-00005
s3://sap-on-aws-ai-ml-demo-ireland/csv/run-1608553911056-part-r-00006
s3://sap-on-aws-ai-ml-demo-ireland/csv/run-1608553911056-part-r-00007
s3://sap-on-aws-ai-ml-demo-ireland/csv/run-1608553911056-part-r-00008
s3://sap-on-aws-ai-ml-demo-ireland/csv/run-1608553911056-part-r-00009
s3://sap-on-aws-ai-ml-demo-ireland/csv/run-1608553911056-part-r-00010
s3://sap-on-aws-ai-ml-demo-ireland/csv/run-1608553911056-part-r-00011
s3://sap-on-aws-ai-ml-demo-ireland/csv/run-1608553911056-part-r-00012
s3://sap-on-aws-ai-ml-demo-ireland/csv/run-1608553911056-part-r-00013
s3://sap-on-aws-ai-m

In [148]:

print("Total number of rows is :- " + str(frame.time.count()))

print ("The data set summery :- "+ str (frame.describe()))

Total number of rows is :- 284807
The data set summery :-               amount          class           time            v1  \
count  284807.000000  284807.000000  284807.000000  2.848070e+05   
mean       88.349619       0.001727   94813.859575  1.759564e-12   
std       250.120109       0.041527   47488.145955  1.958696e+00   
min         0.000000       0.000000       0.000000 -5.640751e+01   
25%         5.600000       0.000000   54201.500000 -9.203734e-01   
50%        22.000000       0.000000   84692.000000  1.810880e-02   
75%        77.165000       0.000000  139320.500000  1.315642e+00   
max     25691.160000       1.000000  172792.000000  2.454930e+00   

                v10           v11           v12           v13           v14  \
count  2.848070e+05  2.848070e+05  2.848070e+05  2.848070e+05  2.848070e+05   
mean   7.095096e-13  1.874828e-12  1.053120e-12  7.127092e-13 -1.476386e-13   
std    1.088850e+00  1.020713e+00  9.992014e-01  9.952742e-01  9.585956e-01   
min   -2.4588

In [134]:
frame.head(100)

,amount,class,time,v1,v10,v11,v12,v13,v14,v15,...,v26,v27,v28,v3,v4,v5,v6,v7,v8,v9
0,8.00,0,138387.0,-0.795056,-1.203517,-0.313037,-0.208788,-0.245882,-0.157083,0.441054,...,1.436251,-0.054368,0.029024,0.583109,-2.785342,2.382625,3.893398,-0.234167,1.134943,0.050420
1,8.00,0,138548.0,0.111859,0.364354,-0.123633,-0.048207,0.143277,0.306454,-0.951601,...,0.553065,0.029222,0.076029,-0.379485,-2.045941,0.902360,-0.387733,0.360705,-0.040848,-1.417019
2,5.49,0,138565.0,-0.042403,-0.245198,1.277904,1.016978,-0.163004,0.341014,-1.145896,...,0.085758,0.243341,0.093153,0.547559,-0.567114,0.184088,-1.119837,0.929956,-0.097869,-0.345062
3,8.00,0,138903.0,1.858781,-1.004874,1.831299,1.201311,0.021144,-1.779012,-0.100576,...,-0.198733,0.067888,-0.032581,-2.180599,0.498997,1.172304,0.586699,-0.012115,0.256517,0.556052
4,8.00,0,138950.0,-1.296529,-1.617568,-1.257854,0.173310,0.106827,0.039036,0.143660,...,0.519343,0.156798,0.049739,0.903022,-2.553679,0.130057,-0.805291,-0.051779,0.537460,0.668344
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,69.99,0,150804.0,2.109244,0.746366,-1.789080,-1.594497,-1.168488,-0.222196,0.499585,...,-0.069085,-0.013351,-0.042937,-1.039291,-1.007770,-0.744493,0.087075,-1.097943,0.085922,0.406713
96,69.99,0,150830.0,1.890816,-0.035017,-1.047794,0.494912,0.626120,-0.361854,0.359532,...,0.590737,-0.032851,-0.030506,-0.476140,0.414250,-0.577885,-0.083524,-0.611221,0.001195,1.198447
97,69.99,0,150951.0,-0.733648,-0.795417,-1.546189,-0.498297,-0.546115,-0.218505,0.077693,...,0.559385,0.086208,0.182649,1.275727,-0.546612,0.395493,-0.129116,0.065601,0.194509,0.485225
98,5.49,0,151073.0,-0.264804,-0.340835,0.604569,0.686511,-0.162737,0.955257,-0.225384,...,-0.273098,0.374493,0.257302,-0.599687,-0.555493,0.495826,-0.580677,0.560749,0.368602,-0.437051
